In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from scipy.optimize import fsolve
from sklearn import linear_model
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score

In [ ]:
#Constant
RT=8.314*298/1000/96.49 #eV

### A. Read and extract from Excel file

In [ ]:
#read excel table
excel=pd.read_excel('GCDFT_compile.xlsx', sheet_name='all_pbed3')
excel.head(7)

In [ ]:
#Extract
valences=excel.to_numpy()[0,1:].astype('float')
pKas=excel.to_numpy()[1,1:].astype('float') #transposed and converted to floats
GpKas=pKas*2.303*8.314*298/1000/96.49 #eV
names=np.array(excel.columns[1:]).astype('str')
#In order of d filling and 3, 4, 5d
metals=['Co','Ni','Cu','Rh','Pd','Ag','Ir','Pt','Au']
ads=[]
efs=[]
for i in range(len(metals)):
    nstart=4+5*i
    nend=7+5*i
    ads.append(excel.to_numpy()[nstart:nend,1:].T.astype('float') )
    efs.append(excel.to_numpy()[nstart-1,:])
ads=np.array(ads)
#Extract Ef
efs=np.array(efs).astype('float')
metalefs=efs[:,0]*27.2114

In [ ]:
#Fit adsorption vs. potential
pots=np.array([0.8,0.4,0.0]) #SHE
fits=[[stats.linregress(pots,i) for i in m] for m in ads]
#Get slope and intercept
gam=np.array([[i.slope for i in m] for m in fits])
Uo=np.array([[-i.intercept/i.slope for i in m] for m in fits])

### B. Langmuir isotherms setup

Input includes:
1. anionChoice: List of anions (excluding any acid-base conjugates).
2. anionc: List of anion concentration.
3. Bulk pH.
4. metalChoice: Metal identity OR metalFeatures: customized feature values for metals (n=6).
5. nsite: List of number of site each anion occupy (based on 111 surface).

Algorithm includes:
1. See if the chosen anions have acid-base conjugates, then compute the conjugates' concentrations based on pH.
2. Compute the $\gamma$ and $U^o$ for all chosen anions and their conjugates.
3. Set up Langmuir ODE and solve.

Acid-base equilibrium\
Assume $A \xleftrightarrow{\text{pKa1}} B \xleftrightarrow{\text{pKa2}} C \xleftrightarrow{\text{pKa3}} ... $ (acid left to base right).

\begin{align}
\frac{[B]}{[A]} &= 10^{\text{pH}-\text{pKa1}} = Q_1 \\
\frac{[C]}{[B]} &= 10^{\text{pH}-\text{pKa2}} = Q_2 \\
[A] + [B] + [C] &= C_0 
\end{align}
Then rearrange this into a system of linear equations:
\begin{align}
-Q_1[A] + [B] &= 0 \\
-Q_2[B] + [C] &= 0 \\
[A] + [B] + [C] &= C_0 
\end{align}
In matrix form of $ax=b$:
\begin{align}
a&=
\begin{bmatrix} 
-Q_1 & 1 & 0 \\
0 & -Q_2 & 1 \\
1 & 1 & 1
\end{bmatrix}\\
b&=
\begin{bmatrix} 
0 \\
0 \\
C_0
\end{bmatrix} 
\hspace{9mm}
x=
\begin{bmatrix} 
[A] \\
[B] \\
[C]
\end{bmatrix}
\end{align}

#### 0. Input

In [ ]:
print(names)
print(metals)

In [ ]:
anionChoice=['HCO3-','NO3-']
initConc=[0.001, 1]
anionSite=[2, 2]
pH=1
metalChoice='Cu'

#### 1. Identify conjugates and solve for concentrations

In [ ]:
#Create conjugates library
conjugates=[['HSO4-', 'SO4--'],
            ['HSO3-', 'SO3--'],
            ['HCO3-', 'CO3--'],
            ['COOHCOO-', 'COOCOO--'],
            ['H2PO4-', 'HPO4--', 'PO4---']]
pKa_lookup=dict(zip(names, pKas)) #lookup table

In [ ]:
#General acid-base equation solver
def solveAcidBase(pKas, pH, initc):
    #Set up system of Henderson-Hasselbalch linear equations
    Qs=[10**(pH - pKa) for pKa in pKas[:-1]] #exclude the last entry 
    #Setup ax=b
    a=np.zeros((len(pKas), len(pKas))) #initialize matrix
    for i in range(len(pKas)-1):
        a[i,i]=-Qs[i] #diagonal elements are K values
        a[i,i+1]=1 #on the right is 1
    a[-1:, :]=1 #last row for intital concentration
    b=np.zeros(len(pKas))
    b[-1]=initc
    #Solve
    concs=np.linalg.solve(a,b)
    return concs

In [ ]:
#Iterate through anionChoice and find conjugates, then solve for concentration in one go
anionConj=[]
concConj=[]
siteConj=[]
flagNO3NO2=False
for a in range(len(anionChoice)):
    anion=anionChoice[a]
    #First determine whether this anion also has other anionic conjugates (i.e., not the neutral acid)
    flagConj=False
    for group in conjugates: 
        for i, entry in enumerate(group):
            if anion==entry: #if found in conjugate library
                flagConj=True
                iSpecies=['acid'] + group #species to solve for including the neutral acid, arranged from acid to base
                pKaSpecies=[pKa_lookup.get(i,'') for i in iSpecies] #lookup pKa
                pKaSpecies=pKaSpecies[1:]+pKaSpecies[:1] #Rearrange to match species and pKa
                #Call solver
                conc=solveAcidBase(pKaSpecies, pH, initConc[a])
                #Save info
                anionConj+=iSpecies[1:] #get the anions, not neutral acid
                concConj+=conc[1:].tolist()
                siteConj+=[anionSite[a]]*len(group)
    if not flagConj: #If has no conjugate anion at all
        iSpecies=['acid'] + [anion] #species to solve for including the neutral acid, arranged from acid to base        
        pKaSpecies=[pKa_lookup.get(i,'') for i in iSpecies] #lookup pKa
        pKaSpecies=pKaSpecies[1:]+pKaSpecies[:1] #Rearrange to match species and pKa
        #Call solver
        conc=solveAcidBase(pKaSpecies, pH, initConc[a])
        #Save info
        anionConj+=iSpecies[1:] #get the anions, not neutral acid
        concConj+=conc[1:].tolist()
        siteConj+=[anionSite[a]]

In [ ]:
#Concat in H and OH
ionList=['H+', 'OH-'] + anionConj
concList=[10**(-pH), 10**(-14+pH)] + concConj
siteList=[1, 1] + siteConj
print(ionList)
print(concList)
print(siteList)

#### 2. Compute $\gamma$ and $U^o$ based on anion list

In [ ]:
#2. Function to output all needed gams and Uos
def GCout(metalChoice, ionList):
    metalInd=metals.index(metalChoice)
    ionInd=[names.tolist().index(i) for i in ionList]
    gams=[gam[metalInd, i] for i in ionInd]
    Uos=[Uo[metalInd, i] for i in ionInd]
    out=list(zip(gams,Uos))
    return np.array(out)

In [ ]:
gamUoList=GCout(metalChoice, ionList)
print(gamUoList)

#### 3. Set up and solve Langmuir ODE

In [ ]:
#Adsorption grand free energy
def omegaAd(gam, Uo, U):
    return gam*(U - Uo)

In [ ]:
def Langmuir(theta, U):
    #adsorption equi constant of all ions
    Kad=[np.exp(-omegaAd(*a, U)/RT) for a in gamUoList]
    #Setup Langmuir equations
    eqns=[]
    for i in range(len(ionList)):
        eqns.append(theta[i+1] - Kad[i]*concList[i]*theta[0]**siteList[i]) #adsorption equilibrium
    eqns.append(np.sum(theta)-1) #site balance
    return eqns

In [ ]:
#Test solve at some U
U=0.845 #V SHE
fsolve(Langmuir, x0=[0.1]*(1+len(ionList)), args=(U)) #choose U

### D. Static plot

In [ ]:
#Define U range
Umin=-1.1
Umax=2.1
USHE=np.arange(Umin,Umax+0.01,0.01) #USHE
#Solve Langmuir at each U
theta=[]
for u in USHE:
    #Solve Langmuir equations
    theta.append(fsolve(Langmuir, x0=[0.1]*(1+len(ionList)), args=(u)))
theta=np.array(theta).T

In [ ]:
#Configure colors
colors=[[232, 174, 74] , [237, 152, 5], #HCO3, CO3
        [25, 180, 43] , [25, 180, 121], [25, 180, 121]] #NO3 & NO2
colors+=[[150-n*10, 180,25] for n in range(10)] #10 carboxylates
colors+=[[180, 25, 170-n*20] for n in range(3)] #3 halides
colors+=[[176, 77, 48], [180, 60, 25], #HSO3, SO3
         [179, 59, 55], [180, 30, 25],  #HSO4, SO4
         [145, 140, 60], [186, 176, 39], [235, 219, 5], #H2PO4, HPO4, PO4
         [14, 178, 207], [27, 194, 12], [14, 178, 207]]
colors=np.array(colors)/255
colors_lookup=dict(zip(names, colors))
colorList=[colors_lookup.get(i) for i in ionList]

In [ ]:
#plot
fig=plt.figure(figsize=(5,2.5),dpi=300)
bottom=np.zeros(len(theta[0]))
for i in range(1,len(theta)):
    plt.fill_between(USHE,theta[i]+bottom,ec='k',zorder=-i, label=ionList[i-1], fc=colorList[i-1])
    bottom +=theta[i]
plt.ylim(0,1)

#plt.legend(bbox_to_anchor=(1.05, 1.05), loc='upper left',fontsize=16)
#plt.xlim(-0.1-0.059*pH, 1.1-0.059*pH) #SHIFT TO RHE
plt.xlim(-0.3, 1)
plt.xticks([-0.2, 0.3, 0.8])
plt.tick_params(axis='both',labelsize=25)
#plt.tick_params(axis='x',labelbottom=False)
#plt.tick_params(axis='y',labelleft=False)
plt.xlabel('Potential (V SHE)',fontsize=22)
plt.ylabel(r'Coverage',fontsize=22)
plt.show()